# Natural Language Processing with Disaster Tweets

competition: https://www.kaggle.com/competitions/nlp-getting-started/overview

code: https://www.kaggle.com/code/faressayah/natural-language-processing-nlp-for-beginners